face emotion recognition


In [24]:
pip install deepface

In [25]:
pip install opencv-contrib-python

In [26]:
# import dependencies/libraries
from deepface import DeepFace
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time


import tensorflow as tf
from tensorflow import keras
from google.colab.patches import cv2_imshow
from datetime import date
import pandas as pd

In [27]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [28]:
# initialize the Haar Cascade face detection model
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

In [29]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [ ]:

# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0
i=0
result_final={}

while i<25:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        continue

    # convert JS response to OpenCV Image
    img = js_to_image(js_reply["img"])
    r=DeepFace.analyze(img , actions=['age', 'gender', 'emotion' ,'race'],enforce_detection=False ,detector_backend = 'opencv')
    result=[]
    for key, val in r[0].items():
      if key=='age' or key=='dominant_emotion' or key=='dominant_race' or key=='dominant_gender':
        result.append(val)

    # create transparent overlay for bounding box
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    # grayscale image for face detection
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # get face region coordinates
    faces = face_cascade.detectMultiScale(gray)
    # get face bounding box for overlay
    font= cv2.FONT_HERSHEY_SIMPLEX
    for (x,y,w,h) in faces:
      bbox_array = cv2.rectangle(bbox_array,(x,y),(x+w,y+h),(255,0,0),2)
      cv2.putText(bbox_array, str(result[0]), (int(x), int(y)), font, 1, (0, 0, 255), 1,cv2.LINE_4)
      cv2.putText(bbox_array, result[1], (int(y)+4, int(w)+4), font, 1, (0, 255, 0), 1)
      cv2.putText(bbox_array, result[2], (int(x)-4, int(h)-4), font, 1, (0, 0, 255), 1)
      cv2.putText(bbox_array, result[3], (int(h)+4, int(x)+4), font, 1, (0, 0, 255), 1,cv2.LINE_4)


    #cv2_imshow(img)
    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    # update bbox so next frame gets new overlay
    bbox = bbox_bytes
    #if cv2.waitKey(2) == ord('q'):  # wait until 'q' key is pressed
        #break
    i=i+1
    result_final[i]=result
print(result_final)


In [31]:
#the final list which contains all the parameters captured from the live steam
print(result_final)


{1: [45, 'Man', 'angry', 'asian'], 2: [36, 'Woman', 'neutral', 'latino hispanic'], 3: [34, 'Woman', 'neutral', 'middle eastern'], 4: [33, 'Woman', 'happy', 'latino hispanic'], 5: [38, 'Woman', 'neutral', 'latino hispanic'], 6: [35, 'Woman', 'neutral', 'indian'], 7: [35, 'Woman', 'neutral', 'latino hispanic'], 8: [33, 'Woman', 'neutral', 'latino hispanic'], 9: [35, 'Woman', 'neutral', 'latino hispanic'], 10: [35, 'Woman', 'neutral', 'latino hispanic'], 11: [34, 'Woman', 'neutral', 'latino hispanic'], 12: [35, 'Woman', 'neutral', 'latino hispanic'], 13: [35, 'Woman', 'neutral', 'latino hispanic'], 14: [33, 'Woman', 'neutral', 'latino hispanic'], 15: [34, 'Woman', 'sad', 'latino hispanic'], 16: [34, 'Woman', 'angry', 'latino hispanic'], 17: [37, 'Woman', 'neutral', 'latino hispanic'], 18: [34, 'Woman', 'sad', 'latino hispanic'], 19: [35, 'Woman', 'neutral', 'latino hispanic'], 20: [35, 'Woman', 'neutral', 'latino hispanic'], 21: [36, 'Woman', 'neutral', 'latino hispanic'], 22: [33, 'Woman

In [32]:
#re-checking the mood detection stored
for k,v in result_final.items():
  print(v[2])

angry
neutral
neutral
happy
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
sad
angry
neutral
sad
neutral
neutral
neutral
neutral
neutral
neutral
neutral


In [33]:
def age_cal(result_final):
  age=0
  for k,v in result_final.items():
    age=age+v[0]
  return(int(age/25))

In [34]:
def gender(result_final):
  count_male=0
  count_female=0
  for k,v in result_final.items():
    if v[1]=='Man':
      count_male=count_male+1
    if v[1]=='Woman':
      count_female=count_female+1
  if count_female>count_male:
    return('Woman')
  else:
    return('Man')

In [35]:
from collections import Counter
def emotion_cal(result_final):
  emotions=[]
  for k,v in result_final.items():
    emotions.append(v[2])
  emotion_counts = Counter(emotions)
  most_common_emotion = emotion_counts.most_common(1)[0][0]
  return (most_common_emotion)



In [36]:
def race(result_final):
  race=[]
  for k,v in result_final.items():
    race.append(v[3])
  race_counts = Counter(race)
  most_common_race= race_counts.most_common(1)[0][0]
  return (most_common_race)

In [37]:
final_list=[]
final_list.append(age_cal(result_final))
final_list.append(gender(result_final))
final_list.append(emotion_cal(result_final))
final_list.append(race(result_final))

In [38]:
#printing the final average parameters
final_list

[35, 'Woman', 'neutral', 'latino hispanic']

In [39]:
#getting the music database
mood_music = pd.read_csv('/content/data_moods (1).csv')
mood_music.head()

,name,album,artist,id,release_date,popularity,length,danceability,acousticness,energy,instrumentalness,liveness,valence,loudness,speechiness,tempo,key,time_signature,mood
0,1999,1999,Prince,2H7PHVdQ3mXqEHXcvclTB0,1982-10-27,68,379266,0.866,0.13700,0.730,0.000000,0.0843,0.625,-8.201,0.0767,118.523,5,4,Happy
1,23,23,Blonde Redhead,4HIwL9ii9CcXpTOTzMq0MP,2007-04-16,43,318800,0.381,0.01890,0.832,0.196000,0.1530,0.166,-5.069,0.0492,120.255,8,4,Sad
2,9 Crimes,9,Damien Rice,5GZEeowhvSieFDiR8fQ2im,2006-11-06,60,217946,0.346,0.91300,0.139,0.000077,0.0934,0.116,-15.326,0.0321,136.168,0,4,Sad
3,99 Luftballons,99 Luftballons,Nena,6HA97v4wEGQ5TUClRM0XLc,1984-08-21,2,233000,0.466,0.08900,0.438,0.000006,0.1130,0.587,-12.858,0.0608,193.100,4,4,Happy
4,A Boy Brushed Red Living In Black And White,They're Only Chasing Safety,Underoath,47IWLfIKOKhFnz1FUEUIkE,2004-01-01,60,268000,0.419,0.00171,0.932,0.000000,0.1370,0.445,-3.604,0.1060,169.881,1,4,Energetic


In [40]:
#calculating the birth year from the age detected
birth_year=int(str(date.today())[0:4])-final_list[0]

mood_music['release_date'] = pd.to_datetime(mood_music['release_date'])
mood_music['Year'] = mood_music['release_date'].dt.year

#filtering the data by adding condition that the song should have release year greater than birth year to have more relevance.
filter_age=mood_music['Year']>=birth_year
filter=mood_music.where(filter_age)
filter=filter.dropna()
filter.reset_index(inplace=True)

#displaying the filtered dataset remaining.
print(filter)

     index                                         name  \
0        1                                           23   
1        2                                     9 Crimes   
2        4  A Boy Brushed Red Living In Black And White   
3        5                             A Burden to Bear   
4        6                                   A La Plage   
..     ...                                          ...   
632    681                                  windcatcher   
633    682              yellow is the color of her eyes   
634    683                           you broke me first   
635    684                          you were good to me   
636    685                                         æfre   

                               album              artist  \
0                                 23      Blonde Redhead   
1                                  9         Damien Rice   
2        They're Only Chasing Safety           Underoath   
3                   A Burden to Bear  Emmanuelle Ri

In [41]:
#now printing the list from remining dataset according to the mood detected.
if(final_list[2]=='angry' or final_list[2]=='disgust' or final_list[2]=='fear' ):

    #for angery,disgust,fear
  filter1=filter['mood']=='Calm'
  f1=filter.where(filter1)
  f1=f1.dropna(subset = ['name','release_date'])
  f1.sort_values('popularity')
  f2 =f1.head(10)
  f2.reset_index(inplace=True)
  print(f2)
if(final_list[2]=='happy' or final_list[2]=='neutral'):
    #for happy, neutral
    filter1=filter['mood']=='Happy'
    f1=filter.where(filter1)
    f1=f1.dropna(subset = ['name','release_date'])
    f1.sort_values('popularity')
    f2 =f1.head(10)
    f2.reset_index(inplace=True)
    print(f2)
if(final_list[2]=='sad'):
       #for Sad
  filter1=filter['mood']=='Sad'
  f1=filter.where(filter1)
  f1=f1.dropna(subset = ['name','release_date'])
  f1.sort_values('popularity')
  f2 =f1.head(10)
  f2.reset_index(inplace=True)
  print(f2)
if(final_list[2]=='surprise'):
     #for surprise
    filter1=filter['mood']=='Energetic'
    f1=filter.where(filter1)
    f1=f1.dropna(subset = ['name','release_date'])
    f1.sort_values('popularity')
    f2 =f1.head(10)
    f2.reset_index(inplace=True)
    print(f2)

   level_0  index                                               name  \
0        5    7.0  A Little Less Conversation - JXL Radio Edit Remix   
1       22   25.0  All or Nothing (feat. Axel Ehnström) - Deluxe Mix   
2       30   33.0                                             Anthem   
3       32   35.0                           Are You Gonna Be My Girl   
4       49   52.0                                     Beautiful Life   
5       54   58.0                                    Better Together   
6       67   71.0                                               Blow   
7       68   72.0                                          Body Rock   
8       71   75.0                                  Bottom Of The Sea   
9       77   81.0                Brimful of Asha - Norman Cook Remix   

                             album            artist                      id  \
0  Elvis 75 - Good Rockin' Tonight     Elvis Presley  4l2hnfUx0esSbITQa7iJt0   
1            Less is More (Deluxe)  Lost Freque